# Zero to Snowflake - Vignette 1: Simple Data Pipeline

このノートブックでは、以下の内容を学習します:
1. 外部ステージからの取り込み
2. 半構造化データとVARIANTデータ型
3. 動的テーブル
4. 動的テーブルによるシンプルなパイプライン
5. 有向非循環グラフ（DAG）によるパイプライン可視化


In [ ]:
-- コンテキストを設定
USE DATABASE tb_101;
USE ROLE tb_data_engineer;
USE WAREHOUSE tb_de_wh;


## 1. 外部ステージからの取り込み

Snowflakeでは、ステージはデータファイルが保存される場所を指定する名前付きデータベースオブジェクトです。


In [ ]:
-- menu_stage ステージを作成
CREATE OR REPLACE STAGE raw_pos.menu_stage
COMMENT = 'メニューデータ用ステージ'
URL = 's3://sfquickstarts/frostbyte_tastybytes/raw_pos/menu/'
FILE_FORMAT = public.csv_ff;

-- menu_stagingテーブルを作成
CREATE OR REPLACE TABLE raw_pos.menu_staging
(
    menu_id NUMBER(19,0),
    menu_type_id NUMBER(38,0),
    menu_type VARCHAR(16777216),
    truck_brand_name VARCHAR(16777216),
    menu_item_id NUMBER(38,0),
    menu_item_name VARCHAR(16777216),
    item_category VARCHAR(16777216),
    item_subcategory VARCHAR(16777216),
    cost_of_goods_usd NUMBER(38,4),
    sale_price_usd NUMBER(38,4),
    menu_item_health_metrics_obj VARIANT
);


In [ ]:
-- ステージから新しいmenu_stagingテーブルにデータをロード
COPY INTO raw_pos.menu_staging
FROM @raw_pos.menu_stage;

-- ロードの成功を確認
SELECT * FROM raw_pos.menu_staging;


## 2. Snowflakeでの半構造化データ

SnowflakeはVARIANTデータ型を使用してJSONなどの半構造化データの処理に優れています。コロン演算子（:）でキー名にアクセスし、角括弧（[]）で配列から要素を選択します。


In [ ]:
-- VARIANT列を確認
SELECT menu_item_health_metrics_obj FROM raw_pos.menu_staging;


In [ ]:
-- VARIANTオブジェクトから原材料を抽出
SELECT
    menu_item_name,
    CAST(menu_item_health_metrics_obj:menu_item_id AS INTEGER) AS menu_item_id,
    menu_item_health_metrics_obj:menu_item_health_metrics[0]:ingredients::ARRAY AS ingredients
FROM raw_pos.menu_staging;


In [ ]:
-- FLATTEN関数を使用してすべての原材料のリストを取得
SELECT
    i.value::STRING AS ingredient_name,
    m.menu_item_health_metrics_obj:menu_item_id::INTEGER AS menu_item_id
FROM
    raw_pos.menu_staging m,
    LATERAL FLATTEN(INPUT => m.menu_item_health_metrics_obj:menu_item_health_metrics[0]:ingredients::ARRAY) i;


## 3. 動的テーブル

動的テーブルはデータ変換パイプラインを簡素化するために設計された強力なツールです。
- 宣言的構文で作成
- 自動データ更新
- データの新鮮さ管理


In [ ]:
-- 原材料用の動的テーブルを作成
CREATE OR REPLACE DYNAMIC TABLE harmonized.ingredient
    LAG = '1 minute'
    WAREHOUSE = 'TB_DE_WH'
AS
    SELECT
    ingredient_name,
    menu_ids
FROM (
    SELECT DISTINCT
        i.value::STRING AS ingredient_name,
        ARRAY_AGG(m.menu_item_id) AS menu_ids
    FROM
        raw_pos.menu_staging m,
        LATERAL FLATTEN(INPUT => menu_item_health_metrics_obj:menu_item_health_metrics[0]:ingredients::ARRAY) i
    GROUP BY i.value::STRING
);

-- ingredient動的テーブルが正常に作成されたことを確認
SELECT * FROM harmonized.ingredient;


In [ ]:
-- 新しいメニュー項目(バインミーサンドイッチ)を追加
INSERT INTO raw_pos.menu_staging 
SELECT 
    10101,
    15,
    'Sandwiches',
    'Better Off Bread',
    157,
    'Banh Mi',
    'Main',
    'Cold Option',
    9.0,
    12.0,
    PARSE_JSON('{
      "menu_item_health_metrics": [
        {
          "ingredients": [
            "French Baguette",
            "Mayonnaise",
            "Pickled Daikon",
            "Cucumber",
            "Pork Belly"
          ],
          "is_dairy_free_flag": "N",
          "is_gluten_free_flag": "N",
          "is_healthy_flag": "Y",
          "is_nut_free_flag": "Y"
        }
      ],
      "menu_item_id": 157
    }'
);


In [ ]:
-- 新しい原材料が表示されることを確認(動的テーブルが更新されるまで最大1分待つ)
SELECT * FROM harmonized.ingredient 
WHERE ingredient_name IN ('French Baguette', 'Pickled Daikon');


## 4. 動的テーブルによるシンプルなパイプライン

ingredient_to_menu_lookup動的テーブルを作成し、原材料とメニュー項目の関連付けを管理します。


In [ ]:
CREATE OR REPLACE DYNAMIC TABLE harmonized.ingredient_to_menu_lookup
    LAG = '1 minute'
    WAREHOUSE = 'TB_DE_WH'    
AS
SELECT
    i.ingredient_name,
    m.menu_item_health_metrics_obj:menu_item_id::INTEGER AS menu_item_id
FROM
    raw_pos.menu_staging m,
    LATERAL FLATTEN(INPUT => m.menu_item_health_metrics_obj:menu_item_health_metrics[0]:ingredients) f
JOIN harmonized.ingredient i ON f.value::STRING = i.ingredient_name;

-- ingredient_to_menu_lookup動的テーブルが正常に作成されたことを確認
SELECT * 
FROM harmonized.ingredient_to_menu_lookup
ORDER BY menu_item_id;


In [ ]:
-- バインミーサンドイッチの注文をシミュレート
INSERT INTO raw_pos.order_header
SELECT 
    459520441,
    15,
    1030,
    101565,
    null,
    200322900,
    TO_TIMESTAMP_NTZ('08:00:00', 'hh:mi:ss'),
    TO_TIMESTAMP_NTZ('14:00:00', 'hh:mi:ss'),
    null,
    TO_TIMESTAMP_NTZ('2022-01-27 08:21:08.000'),
    null,
    'USD',
    14.00,
    null,
    null,
    14.00;
    
INSERT INTO raw_pos.order_detail
SELECT
    904745311,
    459520441,
    157,
    null,
    0,
    2,
    14.00,
    28.00,
    null;


In [ ]:
-- トラック別の原材料使用量をまとめる動的テーブルを作成
CREATE OR REPLACE DYNAMIC TABLE harmonized.ingredient_usage_by_truck 
    LAG = '2 minute'
    WAREHOUSE = 'TB_DE_WH'  
    AS 
    SELECT
        oh.truck_id,
        EXTRACT(YEAR FROM oh.order_ts) AS order_year,
        MONTH(oh.order_ts) AS order_month,
        i.ingredient_name,
        SUM(od.quantity) AS total_ingredients_used
    FROM
        raw_pos.order_detail od
        JOIN raw_pos.order_header oh ON od.order_id = oh.order_id
        JOIN harmonized.ingredient_to_menu_lookup iml ON od.menu_item_id = iml.menu_item_id
        JOIN harmonized.ingredient i ON iml.ingredient_name = i.ingredient_name
        JOIN raw_pos.location l ON l.location_id = oh.location_id
    WHERE l.country = 'United States'
    GROUP BY
        oh.truck_id,
        order_year,
        order_month,
        i.ingredient_name
    ORDER BY
        oh.truck_id,
        total_ingredients_used DESC;


In [ ]:
-- 2022年1月のトラック#15の原材料使用量を表示
SELECT
    truck_id,
    ingredient_name,
    SUM(total_ingredients_used) AS total_ingredients_used,
FROM
    harmonized.ingredient_usage_by_truck
WHERE
    order_month = 1
    AND truck_id = 15
GROUP BY truck_id, ingredient_name
ORDER BY total_ingredients_used DESC;


## 5. 有向非循環グラフ（DAG）によるパイプライン可視化

DAGにアクセスするには:
1. ナビゲーションメニューの「データ」ボタンをクリック
2. 「TB_101」を展開
3. 「HARMONIZED」を展開し、「動的テーブル」を展開
4. 「INGREDIENT」テーブルをクリック


## リセット(必要に応じて実行)


In [ ]:
USE ROLE accountadmin;
-- 動的テーブルを削除
DROP TABLE IF EXISTS raw_pos.menu_staging;
DROP TABLE IF EXISTS harmonized.ingredient;
DROP TABLE IF EXISTS harmonized.ingredient_to_menu_lookup;
DROP TABLE IF EXISTS harmonized.ingredient_usage_by_truck;

-- 挿入を削除
DELETE FROM raw_pos.order_detail
WHERE order_detail_id = 904745311;
DELETE FROM raw_pos.order_header
WHERE order_id = 459520441;

-- Suspend warehouse
ALTER WAREHOUSE tb_de_wh SUSPEND;
